Add project root to path

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load raw data

In [2]:
import pandas as pd

from utils.data.segmentmap import create_from_files
from utils import filename

name = 'dev_l'
segmentmap, allocation_dict = create_from_files(name, regenerate=False)
df = pd.read_csv(filename.data.true(name), sep=' ')

Create dataset objects

In [3]:
from utils.data.generating import create_datacube_set_dict

cube_dim = (32, 32, 32)
empty_cube_dim = (64, 64, 64)
dataset = create_datacube_set_dict(df, filename.data.transformed(name,'power'), segmentmap, allocation_dict, .5, cube_dim, empty_cube_dim, 20)

  0%|          | 0/11112 [00:00<?, ?it/s]

Create dataloader, split to train & test

In [4]:
from utils.data import splitting

train, test = splitting.splitted_loaders(dataset, .8)

Load pretrained 2D model

In [5]:
import segmentation_models_pytorch as smp

model = smp.Unet(encoder_name='resnet18', in_channels=1, classes=1)

Convert pretrained 2D model to 3D

In [6]:
import torch
from models.convert2Dto3D import Conv3dConverter

Conv3dConverter(model, -1, torch.ones(1, 1, 32, 32, 32))

Conv3dConverter(
Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchN

In [7]:
from torch import nn
def loss(y, y_hat):
    yf = y.flatten()
    y_hatf = y_hat.flatten()
    return nn.BCEWithLogitsLoss()(yf, y_hatf)

Create Lightning objects

In [8]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from datetime import datetime


from models.segmentation import Segmenter

version = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
logger = TensorBoardLogger("tb_logs", name="segmenter", version=version)
segmenter = Segmenter(model, loss, train, test, logger)

trainer = pl.Trainer(max_epochs=500, gpus=1, logger=logger)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train!

In [ ]:
trainer.fit(segmenter)

Set SLURM handle signals.

  | Name  | Type | Params
-------------------------------
0 | model | Unet | 42.6 M
-------------------------------
42.6 M    Trainable params
0         Non-trainable params
42.6 M    Total params
/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/core/step_result.py:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)
/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]